In [5]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "sexo_(cod)",
    "prevision_(cod)",
    "hospital_de_procedencia_(cod)",
    "fecha_ingreso",
    "fecha_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "diag_02_secundario_(cod)",
    "diag_02_secundario_(des)",
    "diag_03_secundario_(cod)",
    "diag_03_secundario_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]

In [26]:
# Lee GRD del INT
df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_sabanas["FECHA_INGRESO"] = pd.to_datetime(df_sabanas["FECHA_INGRESO"])
df_sabanas["FECHA_EGRESO"] = pd.to_datetime(df_sabanas["FECHA_EGRESO"])

# Lee GRD Interno del INT
df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv", usecols=COLUMNA_GRD_INTERNO)
# Formatea fechas de GRD
df_grd_interno["fecha_ingreso"] = pd.to_datetime(df_grd_interno["fecha_ingreso"])
df_grd_interno["fecha_de_egreso"] = pd.to_datetime(df_grd_interno["fecha_de_egreso"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_87913/2510431287.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)


In [27]:
# Limpia la base de pabellon, sacando las suspendidas y sin duracion
df_pabellon_limpio = df_pabellon.copy()
df_pabellon_limpio = df_pabellon_limpio.query("suspendida == 'NO' and duracion.notna()")

In [28]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_sabanas.merge(
    df_pabellon_limpio, how="inner", left_on="RUT", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [29]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe()
    .reset_index()
)

In [33]:
# Une la base de GRD Interno y Pabellon segun el RUT del paciente
grd_interno_y_pabellon = df_grd_interno.merge(
    df_pabellon_limpio, how="inner", left_on="rut", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_interno_y_pabellon = grd_interno_y_pabellon.query(
    "fecha >= fecha_ingreso and fecha <= fecha_de_egreso"
)

In [39]:
grd_interno_y_pabellon[
    [
        # "rut",
        "fecha_ingreso",
        "fecha_de_egreso",
        "estancias_totales",
        "diag_01_principal_(des)",
        "proced_01_principal_(cod)",
        "proced_01_principal_(des)",
        "fecha",
        "primer_diagnostico",
        "nombre_de_la_operacion",
        "duracion"
    ]
]

,fecha_ingreso,fecha_de_egreso,estancias_totales,diag_01_principal_(des),proced_01_principal_(cod),proced_01_principal_(des),fecha,primer_diagnostico,nombre_de_la_operacion,duracion
0,2021-12-25,2022-01-03,9,Tumor benigno del timo,7.81,ESCISION TIMO PARCIAL OTRA,2021-12-27,MIASTENIA GRAVIS,TIMECTOMIA X VTC DER,0 days 02:45:00
1,2021-12-23,2022-01-03,11,Estenosis (de la válvula) aórtica,35.22,SUSTITUCION VALVULA AORTICA OTRA,2021-12-27,ESTENOSIS AORTICA SEVERA,REEMPL VALV AORTICO,0 days 04:35:00
3,2021-11-08,2022-01-04,57,"Insuficiencia cardíaca, no especificada",37.51,TRASPLANTE CARDIACO,2021-11-25,ICC ->VALVULOPATIA MITRAL CON INSUF SEVERA,INSTALACION DE PICC LINE,0 days 01:15:00
4,2021-11-08,2022-01-04,57,"Insuficiencia cardíaca, no especificada",37.51,TRASPLANTE CARDIACO,2021-12-11,FALLA CARDIACA,ECMO VA FEM,0 days 02:10:00
6,2021-11-08,2022-01-04,57,"Insuficiencia cardíaca, no especificada",37.51,TRASPLANTE CARDIACO,2021-12-16,PROCURAMIENTO,PROCURAMIENTO,0 days 03:30:00
...,...,...,...,...,...,...,...,...,...,...
33075,2017-11-26,2017-12-30,34,Ruptura de aneurisma de la aorta toracoabdominal,38.34,RESECCION AORTA CON ANASTOMOSIS,2017-11-29,HEMORRAGIA POST OP S/P TORACOABDOMINAL,REEXPLORACION X SANGRADO,0 days 02:15:00
33077,2017-10-05,2017-12-31,87,Insuficiencia cardíaca congestiva,37.51,TRASPLANTE CARDIACO,2017-11-14,MIOCARDIOPATIA DILATADA,CONEXIÓN ASISTENCIA VENTRICULAR,0 days 02:40:00
33078,2017-10-05,2017-12-31,87,Insuficiencia cardíaca congestiva,37.51,TRASPLANTE CARDIACO,2017-12-11,INSUF CARDIACA,TRASPLANTE CARDIACO,0 days 08:40:00
33079,2017-10-05,2017-12-31,87,Insuficiencia cardíaca congestiva,37.51,TRASPLANTE CARDIACO,2017-12-11,PROCURAMIENTO,PROCURAMIENTO,0 days 06:40:00
